In [1]:
#-*- coding : utf-8 -*-
# coding: utf-8

%load_ext autoreload
%autoreload 2


# 0. what for
1. 用于infer GT-2的params
# 1. preparations
## 1.1 全局设置

In [2]:
import pandas as pd
import numpy as np
import math
import os
import random
import torch.utils.data
import mydataset_GT2
from functorch import vmap
from mydataset_GT2 import myDataset
import torch.nn as nn
from torch.utils.data import DataLoader, SubsetRandomSampler
import torch.nn.functional as F
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
# from tensorboardX import SummaryWriter
from visdom import Visdom
from graphviz import Digraph
import torchvision
from torchviz import make_dot
from torch.nn.utils.rnn import pad_sequence
import pygraphviz as pgv
import networkx as nx
import matplotlib.pyplot as plt
import geomloss
import time
# from GT_model.GT_2.SA_for_PT_funcs_delta_eq1 import *
from GT_model.GT_2 import SA_for_PT_funcs_delta_eq1
from importlib import reload
from functools import partial
# import logging
# import pytorch_warmup as warmup
# logging.basicConfig(filename="loss_info",filemode="w",level=logging.DEBUG)
# logger = logging.getLogger(__name__)
from torch.cuda.amp import GradScaler,autocast
scaler = GradScaler(backoff_factor = 0.1)
import scipy
from torch.autograd.gradcheck import gradcheck

from Config import config_GT2
import loss
import plot

# import GT_model.GT_2.SA_for_PT_funcs_delta_eq1
# from GT_model.GT_2.SA_for_PT_funcs_delta_eq1 import *
torch.set_default_tensor_type(torch.FloatTensor)
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

## 1.2 Global settings and reloading

In [4]:
reload(config_GT2)  # 必须reload！！
reload(loss)    # 必须reload！！
reload(plot)
reload(mydataset_GT2)
reload(SA_for_PT_funcs_delta_eq1)


# from loss import validate
from plot import plot_net
from plot import plot_alpha
from plot import plot_labda

opt = config_GT2.DefaultConfig()

total_train_step = 0
total_test_step = 0


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
print(device)

bound_alpha = torch.tensor([-0.3,0.3],device=device)
bound_labda = torch.tensor([0.01,18],device=device)

cuda:0


# 2. Dataset and Dataloader
1. DataLoader中的`shuffer=True`表示在每一次epoch中都打乱所有数据的顺序，然后以batch为单位从头到尾按顺序取用数据。这样的结果就是不同epoch中的数据都是乱序的,设置随机种子的作用就是让你的每一次训练都乱的一样，

## 2.1 Dataset and spliting


In [6]:
target_metric_list=[]
data_1 = [1,2,3]
target_metric_list.append(torch.tensor(data_1, dtype=torch.int64))
target_metric_tensor = torch.stack(target_metric_list).float()
del target_metric_list
target_metric_tensor

tensor([[1., 2., 3.]])

In [4]:
def setup_seed(seed):
    torch.manual_seed(seed)
    # torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    # torch.backends.cudnn.deterministic = True

seed = 26
setup_seed(seed)

- 读取data

In [5]:
dataset = myDataset(opt.train_path, opt.target_path_metric,opt.target_path_loss, opt.params_opitim_path, opt.data_key_path, opt.NLL_metric_path)

- 产生index的乱序排列

In [6]:
DATA_len = dataset.__len__()
shuffled_indices = np.random.permutation(DATA_len)

train_idx = shuffled_indices[:int(opt.train_pct*DATA_len)]
tmp = int((opt.train_pct+opt.vali_pct)*DATA_len)
val_idx = shuffled_indices[int(opt.train_pct*DATA_len):tmp]
test_idx = shuffled_indices[tmp:]

print(train_idx)

[ 493  129  826  231  500  334 1244  835  580  989  479  419  804  670
   69  674  611  894  276 1049  153  425  321  935  782 1170 1069 1037
  478  434 1203 1220  636    4  262  682  516 1097 1082   63  898 1204
 1239  812  716   50  272  846 1199 1116  596  111  901  925  687 1180
  986  363  311  664  273  730 1292  737  396  190  285 1241  338 1096
  693 1190 1034 1281 1171  414  348  232  532  535 1166   32  483  926
  953 1163  110  562 1173  374  341 1025  697 1168  105 1031 1035  726
   93 1086  771  827  579 1129   74 1238  328 1023  767 1211   44  843
  116 1104  678  108  174  976  359 1046  983  457  647 1134  654  159
  604 1164 1277 1209  447  605  526  234  946  969  295 1028  191  905
  442 1218  961  233   57  563  757 1274  863  630  489 1193  367  164
 1059  155 1078  525  197  181  281 1165  956   17  947  623 1206  777
  732  797  984  227  634  944  606  199  899  426  375  783  954 1077
  924 1074 1039  694  666 1118  149  655  286  465  381  798  909  758
  218 

- 根据这个乱序排列抽取dataset

## 2.2 Dataloader and collating
1. 主要是对label数据进行collate
    - 按照batch中的最大target data长度进行padding，padding with 0
2. 返回的结果多一个batch dim,比如下面的`5`
    - After collating:
        - `torch.Size([5, 3, 300]),torch.Size([5, 87, 2])`
        - `87`是最长的targets data长度

In [7]:
def my_collate_fn(data):
# 这里的data是一个list， list的元素是元组: (self.data, self.label)
# collate_fn的作用是把[(data, label),(data, label)...]转化成([data, data...],[label,label...])
# 假设self.data的一个data的shape为(channels, length), 每一个channel的length相等,
# data[索引到index(batch)][索引到data或者label][索引到channel]

    data_list = []              # training data
    target_metric_list = []     # target data for computing metric of NN
    target_loss_list = []       # target data for computing loss of NN, (ls T)

    target_params_list = []       # target data for computing loss of NN, (TARGET=5 or 1)
    setting_list = []
    metric_list = []

    data_len = len(data)        # 读进来的data batch的大小

    batch = 0

    while batch < data_len:
        data_list.append(torch.tensor(data[batch][0]))
        # target_data只保留N这一列
        # print("target[0] shape:",torch.tensor(data[batch][1][:,0]).shape)
        target_metric_list.append(torch.tensor(data[batch][1][:,0], dtype=torch.int64))
        target_loss_list.append(torch.tensor(data[batch][2][:,0], dtype=torch.int64))
        target_params_list.append(torch.tensor(data[batch][3]))
        setting_list.append(torch.tensor(data[batch][4]))
        metric_list.append(torch.tensor(data[batch][5]))
        batch += 1


    # Pad target data with zeros
    target_metric_padded = pad_sequence(target_metric_list,batch_first=True)
    target_loss_padded = pad_sequence(target_loss_list,batch_first=True)
    target_metric_tensor = target_metric_padded.float()
    target_loss_padded = target_loss_padded.float()

    data_tensor = torch.stack(data_list).float()
    target_params_tensor = torch.stack(target_params_list).float()
    setting_tensor = torch.stack(setting_list).float()
    metric_tensor = torch.stack(metric_list).float()


    return data_tensor, target_metric_tensor, target_loss_padded, target_params_tensor, setting_tensor, metric_tensor

### 2.2.4 DataLoader

In [8]:
train_loader = DataLoader(dataset = dataset,batch_size = 10, shuffle=False, num_workers=0, drop_last=False, sampler=SubsetRandomSampler(train_idx), collate_fn = my_collate_fn)

val_loader = DataLoader(dataset = dataset,batch_size = 10, shuffle=False, num_workers=0, drop_last=False, sampler=SubsetRandomSampler(val_idx),collate_fn = my_collate_fn)

test_loader = DataLoader(dataset = dataset,batch_size = 1, shuffle=False, num_workers=0, drop_last=False, sampler=SubsetRandomSampler(test_idx),collate_fn = my_collate_fn)

# 3. The Net and Init
1. BatchNorm1d: The mean and std are calculated per-dimension over the mini-batches
## 3.2 MLP-1
### 3.2.1 1层MLP
1. output dim=2，一层线性层

In [9]:
# 参数量34
# 1层MLP+MDN
class MLP_1_1(nn.Module):
    # code->generate->override methods
    def __init__(self) -> None:
        super().__init__()
        self.BN1 = nn.BatchNorm1d(num_features=8,affine=True)

        self.block_alpha = nn.Sequential(
            nn.Linear(8, 1),
            nn.Tanh()
        )
        self.block_labda = nn.Sequential(
            nn.Linear(8, 1),
            nn.ReLU()
        )

    def forward(self, x):
        x = torch.squeeze(x,dim=1)
        x = self.BN1(x)

        alpha = self.block_alpha(x)
        labda = self.block_labda(x)

        # Clamp
        alpha = torch.clamp(alpha,min=bound_alpha[0],max=bound_alpha[1])
        labda = torch.clamp(labda,min=bound_labda[0],max=bound_labda[1])

        # alpha.register_hook(save_grad("alpha"))
        # labda.register_hook(save_grad("labda"))
        return alpha,labda

In [10]:
mlp = MLP_1_1()
mlp = mlp.to(device=device)
summary(mlp, (1,8))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm1d-1                    [-1, 8]              16
            Linear-2                    [-1, 1]               9
              Tanh-3                    [-1, 1]               0
            Linear-4                    [-1, 1]               9
              ReLU-5                    [-1, 1]               0
Total params: 34
Trainable params: 34
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


### 3.2.2 二层

In [11]:
# 98
# 1层MLP+MDN
class MLP_1_2(nn.Module):
    # code->generate->override methods
    def __init__(self) -> None:
        super().__init__()
        self.BN1 = nn.BatchNorm1d(num_features=8,affine=True)

        self.block_alpha = nn.Sequential(
            nn.Linear(8, 4),
            nn.Tanh(),
            nn.Linear(4, 1),
            nn.Tanh()
        )
        self.block_labda = nn.Sequential(
            nn.Linear(8, 4),
            nn.ReLU(),
            nn.Linear(4, 1),
            nn.ReLU()
        )

    def forward(self, x):
        x = torch.squeeze(x,dim=1)
        x = self.BN1(x)

        alpha = self.block_alpha(x)
        labda = self.block_labda(x)

        # Clamp
        alpha = torch.clamp(alpha,min=bound_alpha[0],max=bound_alpha[1])
        labda = torch.clamp(labda,min=bound_labda[0],max=bound_labda[1])
        return alpha,labda

In [12]:
mlp = MLP_1_2()
mlp = mlp.to(device=device)
summary(mlp, (1,8))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm1d-1                    [-1, 8]              16
            Linear-2                    [-1, 4]              36
              Tanh-3                    [-1, 4]               0
            Linear-4                    [-1, 1]               5
              Tanh-5                    [-1, 1]               0
            Linear-6                    [-1, 4]              36
              ReLU-7                    [-1, 4]               0
            Linear-8                    [-1, 1]               5
              ReLU-9                    [-1, 1]               0
Total params: 98
Trainable params: 98
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
-----------------------------------------------------

# 4. Loss
## 4.1 grad hooks


In [11]:
############# Tensor #############:
def C(t,b):
    return 0.2*t*b

def f_ts(x, alpha, device):
    # -alpha*x不能超过85，否则overflow
    y1 = torch.clamp(-alpha*x,max=torch.tensor(85.,device=device))#.to(device)       # torch.exp(x), x<85
    y2 = (1-torch.exp(y1))  #.to(device)

    return y2

def f_2_ts(x, alpha,device):
    # -alpha*x不能超过85，否则overflow
    y1 = torch.min(-alpha*x,torch.tensor(85.)).to(device)       # torch.exp(x), x<85
    y2 = (torch.exp(y1))#.to(device)

    return y2

In [12]:
# root = U_{t-1}
def f_Equi_ts(t, T, v, d, b, alpha, labda, device):

    tmp = (v-d*(t-1)-C((t-2),b) - b).to(device)

    if (tmp>0):

        root = (labda*f_ts(C(t-2,b),alpha,device) + f_ts(((v-d*(t-1)-C((t-2),b) - b)),alpha,device)) / (labda*f_ts(C(t-2,b)+b,alpha,device) + f_ts(((v-d*(t-1)-C((t-2),b) - b)),alpha,device))

    else:

        root = (1- f_2_ts(-(v-(t-1)*d-b),alpha,device)) / (f_2_ts(b,alpha,device) - f_2_ts(-(v-(t-1)*d-b),alpha,device))

    assert not torch.isnan(root.detach()),f"root = {root.detach()} and t, T, v, d, b, alpha, labda = {t, T, v, d, b, alpha.detach(), labda.detach()}"
    return root

# 返回log(root)
def f_Equi_ts_log(t, T, v, d, b, alpha, labda, device):

    tmp = (v-d*(t-1)-C((t-2),b) - b).to(device)

    if (tmp>0):

        root = torch.log(torch.abs((labda*f_ts(C(t-2,b),alpha,device) + f_ts(((v-d*(t-1)-C((t-2),b) - b)),alpha,device)))) \
            - torch.log(torch.abs((labda*f_ts(C(t-2,b)+b,alpha,device) + f_ts(((v-d*(t-1)-C((t-2),b) - b)),alpha,device))))

    else:
        # 分母!=0
        # root = torch.log(torch.abs((1- f_2_ts(-(v-(t-1)*d-b),alpha,device)))) \
        #     - (-alpha*b)- torch.log(torch.abs((1 - f_2_ts(-(v-(t-1)*d),alpha,device))))

        root = torch.log(torch.abs((1- f_2_ts(-(v-(t-1)*d-b),alpha,device)))) \
            - torch.log(torch.abs((f_2_ts(b,alpha,device) - f_2_ts(-(v-(t-1)*d-b),alpha,device))))

    assert not torch.isnan(root.detach()),f"root = {root.detach()}, tmp = {tmp}, and t, T, v, d, b, alpha, labda = {t, T, v, d, b, alpha.detach(), labda.detach()}"
    assert not torch.isinf(root.detach()),f"root = {root.detach()}, tmp = {tmp}, and t, T, v, d, b, alpha, labda = {t, T, v, d, b, alpha.detach(), labda.detach()}"

    return root


def f_Equi_ts_log_vmap(t, v, d, b, alpha, labda):       # , device
    root = torch.where((v-d*(t-1)-C((t-2),b) - b)>0,
                        torch.log(torch.abs(
                                (labda*f_ts(C(t-2,b),alpha,device) + f_ts(((v-d*(t-1)-C((t-2),b) - b)),alpha,device)) )) \
                        - torch.log(torch.abs(
                                (labda*f_ts(C(t-2,b)+b,alpha,device) + f_ts(((v-d*(t-1)-C((t-2),b) - b)),alpha,device)) )),
                        torch.log(torch.abs(
                            (1- f_2_ts(-(v-(t-1)*d-b),alpha,device)) )) \
                        - torch.log(torch.abs((f_2_ts(b,alpha,device) - f_2_ts(-(v-(t-1)*d-b),alpha,device)) ))
                        )
    return root

In [13]:

def save_grad(name):
    print("****")
    def hook(grad):
        print(f"name={name}, grad={grad}")
    return hook


def hook_forward_fn(module, input, output):
    print("It's forward: ")
    print(f"module: {module}")
    print(f"input: {input}")
    print(f"output: {output}")
    print("="*20)

def hook_backward_fn(module, grad_input, grad_output):
    print("It's backward: ")
    print(f"module: {module}")
    print(f"grad_input: {grad_input}")
    print(f"grad_output: {grad_output}")
    print("="*20)


## 4.2 get U and LEN,T
- 注意T的逻辑：
    - 1.首先计算T。而且target data中没有包含>T的部分
    - 2.再之，计算LEN: LEN=max(target_T)，没必要计算多余的U[t]

In [14]:
def get_LEN_T_ts(v,b,d, max_T):
    """

    Args:
        v:
        b:
        d:
        max_T:

    Returns:
        LEN: Length of U vector
        T: max possible duration: max_T or theoratic T

    """

    if d == 0:          # fixed-price
        # T = np.inf
        T = max_T       # 最多计算target data需要的duration
    else:               # asc-price
        T = torch.floor((v - b) / d)

    LEN = max_T
    return LEN,T

In [15]:
def get_U_GT2_ts(LEN, T,v, d, b, alpha, labda, eps = 1e-30, device = device):

    # U: the prob. that someone offers a bid in t_th round
    # U_i = torch.tensor([eps] * (int(LEN.detach().cpu()) + 2),device=device)
    # U_i[0], U_i[1] = torch.tensor(0.), torch.tensor(1.)  # u[0]用不到，设为1,u[1]=1保证auction至少1轮
    # idx = torch.arange(2,len(U_i))
    # U_i[2:] = torch.tensor([f_Equi_ts(t, T, v, d, b, alpha, labda,device) for t in idx])

    U_head = torch.tensor([1.,1.],requires_grad=True,device=device)  # u[0]用不到,u[1]=1保证auction至少1轮

    idx = torch.arange(2,LEN+1)
    for t in idx:
        # 记录一下，work了。。。。
        if t <= T:
            U_head = torch.cat([U_head, f_Equi_ts(t, T, v, d, b, alpha, labda,device)])
        else:   # 计算metric才需要这个else
            U_head = torch.cat([U_head,torch.tensor([eps],requires_grad=True,device=device)])
    # 这样写不work？？but why
    # U_tail = torch.tensor([f_Equi_ts(t, T,v, d, b, alpha, labda,device) for t in idx],requires_grad=True)#.to(device)
    # alpha.register_hook(save_grad("alpha"))       # OK
    # labda.register_hook(save_grad("labda"))       # OK
    U_T_1 = torch.tensor([eps],requires_grad=True,device=device)  # u[T+1]表示最后拍卖在T+1轮发生的概率为0
    # U_head.register_hook(save_grad("U_head"))
    # print("U_head:",U_head)
    U_i = torch.concat([U_head,U_T_1])#.to(device)     # 减少原地修改，使用concat拼接
    U_i = U_i.to(device)

    # eps < U < 1-eps
    # 实际上在pytorch中 torch.tensor(1-eps)=1.
    U = torch.clip(U_i,torch.tensor(eps,device=device),torch.tensor(1-eps,device=device))
    return U

- 返回U_log

In [16]:
def get_U_GT2_ts_log(LEN, T, v, d, b, alpha, labda, eps = 1e-30, device = device):

    # U: the prob. that someone offers a bid in t_th round
    # U_i = torch.tensor([eps] * (int(LEN.detach().cpu()) + 2),device=device)
    # U_i[0], U_i[1] = torch.tensor(0.), torch.tensor(1.)  # u[0]用不到，设为1,u[1]=1保证auction至少1轮
    # idx = torch.arange(2,len(U_i))
    # U_i[2:] = torch.tensor([f_Equi_ts(t, T, v, d, b, alpha, labda,device) for t in idx])

    # log(1) = 0.
    # U_head = torch.tensor([0.,0.],requires_grad=True,device=device)  # u[0]用不到,u[1]=1保证auction至少1轮
    U_head = torch.tensor([0.,0.],requires_grad=True)  # u[0]用不到,u[1]=1保证auction至少1轮

    idx = torch.arange(2,LEN+1).to(device)

    # 方法一：
    for t in idx:
        # 记录一下，work了。。。。
        U_head = torch.cat([U_head, f_Equi_ts_log(t, T, v, d, b, alpha, labda,device)])

    # 这样写不work？？but why
    # U_tail = torch.tensor([f_Equi_ts(t, T,v, d, b, alpha, labda,device) for t in idx],requires_grad=True)#.to(device)
    # alpha.register_hook(save_grad("alpha"))       # OK
    # labda.register_hook(save_grad("labda"))       # OK
    U_T_1 = torch.log(torch.tensor([eps],requires_grad=True,device=device))  # u[T+1]表示最后拍卖在T+1轮发生的概率为0
    # U_head.register_hook(save_grad("U_head"))
    # print("U_head:",U_head)
    U_i_log = torch.concat([U_head,U_T_1])#.to(device)     # 减少原地修改，使用concat拼接

    U_i_log = U_i_log.to(device)

    # eps < U < 1-eps
    # 实际上在pytorch中 torch.tensor(1-eps)=1.
    # U = torch.clip(U_i,torch.tensor(eps,device=device),torch.tensor(1-eps,device=device))
    return U_i_log

- 返回U_log，并且使用vmap：
    - 弊端，device是个bug，目前没办法传

In [17]:
def get_U_GT2_ts_log_vmap(LEN, T, v, d, b, alpha, labda, eps = 1e-30, device = device):

    # U: the prob. that someone offers a bid in t_th round
    # log(1) = 0.
    U_head = torch.tensor([0.,0.],requires_grad=True,device=device)  # u[0]用不到,u[1]=1保证auction至少1轮

    idx = torch.arange(2,LEN+1).to(device)

    # 方法二：使用vmap：
    v_extend = torch.expand_copy(v,size=idx.shape).to(device)    # 必须扩张到和idx同一个size否则报错
    b_extend = torch.expand_copy(b,size=idx.shape).to(device)
    d_extend = torch.expand_copy(d,size=idx.shape).to(device)
    alpha_extend = torch.expand_copy(alpha,size=idx.shape).to(device)
    labda_extend = torch.expand_copy(labda,size=idx.shape).to(device)

    f_Equi_vec = vmap(f_Equi_ts_log_vmap)  # [N, D], [N, D] -> [N]
    U_root = f_Equi_vec(idx, v_extend, d_extend, b_extend, alpha_extend, labda_extend)

    U_T_1 = torch.log(torch.tensor([eps],requires_grad=True,device=device))  # u[T+1]表示最后拍卖在T+1轮发生的概率为0
    U_i_log = torch.concat([U_head,U_root,U_T_1])                            # 减少原地修改，使用concat拼接

    U_i_log = U_i_log.to(device)

    # eps < U < 1-eps
    # 实际上在pytorch中 torch.tensor(1-eps)=1.
    # U = torch.clip(U_i,torch.tensor(eps,device=device),torch.tensor(1-eps,device=device))

    del idx
    del v_extend
    del b_extend
    del d_extend
    return U_i_log

In [18]:
# 求和从1到target[i]
# def get_sum_1_vmap(U_log_cumsum, target):


## 4.3 NLL loss
1. loss 用于处理NLL loss
2. loss function:

- 注意凡是计算中会出现nan（root求解，log of U的求解等，都已经用`assert`提示），因此出现nan问题，一定不是数值计算的问题

In [19]:
def loss_fn_2(input_data, Alpha, Labda, Target_data, eps, device):
    # 注意：P[i][t] = U[i][1]*U[i][2]*...*(1-U[i][t+1])
    loss_sum = torch.tensor(0.,device=device,requires_grad=True)
    for i in range(len(input_data)):

        # Get target data
        target = Target_data[i, :].long()

        # Solve for U from Equi. condt.
        # Get settings
        setting = input_data[i, :, 0:3]
        d = setting[0, 0]
        b = setting[0, 1]
        v = setting[0, 2]

        # 1.首先计算LEN,T
        LEN,T = get_LEN_T_ts(v,b,d,max(target))
        # 2.然后筛选target data：大于T的没必要，算不了

        idx = torch.nonzero(target)
        target_nonzero = target[idx]  #.reshape(1,-1).squeeze().long()      # int

        # U_i最大必须可以访问到U_i[LEN+1],因为计算P[LEN]需要用到U[LEN+1]

        # 方法一：先求U再求U_log
        U = get_U_GT2_ts(LEN, T, v, d, b, Alpha[i,:], Labda[i,:], eps = eps, device=device)
        assert not (torch.any(torch.isnan(U.detach()))), f"U has NaN and U = {U.detach()}, and It should NOT be NaN actually"
        # U.register_hook(save_grad('U'))
        # U_i最大必须可以访问到U_i[LEN+1],因为计算P[LEN]需要用到U[LEN+1]
        assert max(target_nonzero) + 2 == len(U.detach()),"U长度不正确"

        U_log = torch.log(U)#.to(device)
        assert not (torch.any(torch.isnan(U_log.detach()))), f"U_log has NaN and U_log = {U_log.detach()}, and U = {U.detach()}"

        U_log_cumsum = torch.cumsum(U_log,dim=0)#.to(device)
        # assert U_log_cumsum.shape == U.shape, "U_log_cumsum.shape != U.shape"

        # U_log_cumsum_extend.shape: [len(target_nonzero), LEN]
        U_log_cumsum_extend = torch.repeat_interleave(U_log_cumsum[None,:], repeats=len(target_nonzero),dim=0)#.to(device)

        # print("U_log_cumsum_extend.shape shoule be: [len(target_nonzero),len(U)]")
        # U_log_cumsum.register_hook(save_grad('U_log_cumsum'))

        U_sum_1_idx = target_nonzero
        U_sum_2_idx = target_nonzero.squeeze()+1
        U_sum_1 = torch.gather(U_log_cumsum_extend,dim=1,index=U_sum_1_idx)#.to(device)
        U_sum_2 = torch.log(torch.max(1-torch.gather(U,0,index=U_sum_2_idx),torch.tensor(eps,device=device)))#.to(device)
        loss_sum  = U_sum_1.sum() + U_sum_2.sum() + loss_sum
        assert not torch.isnan(U_sum_1.detach().sum()),"U_sum_1=nan"
        assert not torch.isnan(U_sum_2.detach().sum()),"U_sum_1=nan"
        # loss_sum.register_hook(save_grad('loss_sum'))

    # 除以batchsize
    # loss_sum.register_hook(save_grad('loss_sum'))
    # print("ALPHA:register_hook")
    # handler = Alpha.register_hook(save_grad('Alpha'))

    return -loss_sum/len(input_data)

- 直接得到U_log
- 忽略U_sum_2？

In [20]:
def loss_fn_3(input_data, Alpha, Labda, Target_data, eps, device):
    # 注意：P[i][t] = U[i][1]*U[i][2]*...*(1-U[i][t+1])
    loss_sum = torch.tensor(0.,device=device,requires_grad=True)
    for i in range(len(input_data)):

        # Get target data
        target = Target_data[i, :].long()

        # Solve for U from Equi. condt.
        # Get settings
        setting = input_data[i, :, 0:3]
        d = setting[0, 0]
        b = setting[0, 1]
        v = setting[0, 2]

        # 1.首先计算LEN,T
        LEN,T = get_LEN_T_ts(v,b,d,max(target))
        # 2.然后筛选target data：大于T的没必要，算不了

        idx = torch.nonzero(target)
        target_nonzero = target[idx]  #.reshape(1,-1).squeeze().long()      # int

        # U_i最大必须可以访问到U_i[LEN+1],因为计算P[LEN]需要用到U[LEN+1]
        # 方法二：直接求U_log，并且使用vmap进行并行运算
        U_log = get_U_GT2_ts_log_vmap(LEN, T, v, d, b, Alpha[i,:], Labda[i,:], eps = eps, device=device)
        # assert not (torch.any(torch.isnan(U_log.detach()))), f"U_log has NaN and U_log = {U_log.detach()}"

        U_log_cumsum = torch.cumsum(U_log,dim=0)#.to(device)
        # assert U_log_cumsum.shape == U.shape, "U_log_cumsum.shape != U.shape"

        # U_log_cumsum_extend.shape: [len(target_nonzero), LEN]

        # 减少显存的方法一：
        # 无非是求和from 1 to i in target_nonzero
        # U_sum_1 = get_sum_1_vmap(U_log_cumsum,target_nonzero)

        # 通过复制的方法二：
        U_log_cumsum_extend = torch.repeat_interleave(U_log_cumsum[None,:], repeats=len(target_nonzero),dim=0)#.to(device)

        # print("U_log_cumsum_extend.shape shoule be: [len(target_nonzero),len(U)]")
        # U_log_cumsum.register_hook(save_grad('U_log_cumsum'))

        U_sum_1_idx = target_nonzero
        U_sum_2_idx = target_nonzero.squeeze()+1
        U_sum_1 = torch.gather(U_log_cumsum_extend,dim=1,index=U_sum_1_idx)#.to(device)
        # U_sum_2 = torch.log(torch.max(1-torch.gather(U,0,index=U_sum_2_idx),torch.tensor(eps,device=device)))#.to(device)
        # U_sum_2 = 1-torch.gather(U_log,0,index=U_sum_2_idx)#.to(device)
        # 或许可以：用log(U[t+1])表示log(1-U[t+1])
        # 潜在的问题：gather之后的值太小，因此exp之后非常接近1，用1-去之后是一个接近0的值，log=inf
        U_sum_2 = torch.log(torch.max(1-torch.exp(torch.gather(U_log,0,index=U_sum_2_idx)),torch.tensor(opt.MIN_LOSS)))#.to(device)
        # print(f"torch.gather(U_log,0,index=U_sum_2_idx)={torch.gather(U_log,0,index=U_sum_2_idx)}")
        loss_sum = U_sum_1.sum() + U_sum_2.sum() + loss_sum

        # print(f"old method: U_sum_1.sum() ={U_sum_1.sum().detach().cpu()}, U_sum_2.sum()={U_sum_2.sum().detach().cpu()}")

        # assert not torch.isnan(U_sum_1.detach().sum()),f"U_sum_1={U_sum_1}"
        # assert not torch.isinf(U_sum_1.detach().sum()),f"U_sum_1={U_sum_1}"
        # assert not torch.isnan(U_sum_2.detach().sum()),f"U_sum_2={U_sum_2}"
        # assert not torch.isinf(U_sum_2.detach().sum()),f"U_sum_2={U_sum_2}"
        # loss_sum.register_hook(save_grad('loss_sum'))

    # 除以batchsize
    # loss_sum.register_hook(save_grad('loss_sum'))
    # print("ALPHA:register_hook")
    # handler = Alpha.register_hook(save_grad('Alpha'))
    return -loss_sum/len(input_data)

## 4.3 validation

In [22]:
def validate_params(mlp, test_loader, eps, device):

    loss_sum = torch.tensor(0., device=device, requires_grad=False)
    # GT_metric = torch.tensor([0.,0.]).reshape(1,2)
    GT_metric = torch.tensor([0.,0.,0.,0.,0.]).reshape(1,-1)
    cnt = len(test_idx)
    for batch_id, data in enumerate(test_loader):

        input_data, target_metric,_, target_params_data, _, metric_data= data

        # print(f"---- {batch_id} batch----")
        # Do the inference
        input_data = input_data.to(device)
        target_data = target_metric.to(device)
        # target_params_data = target_params_data.to(device)      # params inferred by SA

        Alpha, Labda = mlp(input_data)

        Alpha = Alpha.detach().cpu().numpy()
        Labda = Labda.detach().cpu().numpy()

        GT_metric += torch.sum(metric_data,dim=0)
        # print(f"Alpha, Labda = {Alpha, Labda}")
        for i in range(len(Alpha)):
            # Get target data
            target = target_data[i, :]
            idx = torch.nonzero(target)
            target_nonzero = target[idx].detach().cpu().squeeze().numpy()
            target_ls = [int(x) for x in target_nonzero]        # 统一一下，target在这里是int list

            # Solve for U from Equi. condt.
            # Get settings
            setting = input_data[i, :, 0:3]
            d = setting[0, 0].detach().cpu().numpy().item()
            b = setting[0, 1].detach().cpu().numpy().item()
            v = setting[0, 2].detach().cpu().numpy().item()

            #### metric计算参考自 GT_2_1
            LEN,T =  SA_for_PT_funcs_delta_eq1.get_LEN_T(v,b,d,max(target_ls))

            # Solve for U
            U = SA_for_PT_funcs_delta_eq1.get_U_GT2(LEN,v,d,b,Alpha[i].item(),Labda[i].item(),eps=0.)

            # 返回值是正值
            nll_metric = SA_for_PT_funcs_delta_eq1.get_nll_meric(target_ls, U, LEN,TARGET = 1)

            loss_sum = nll_metric + loss_sum

    return loss_sum / cnt, GT_metric/ cnt

# 5. Plot
## 5.4 plot the loss
1. 直接根据参数决定draw什么loss，反正loss都画在一个window里面

In [23]:
env_str = "params_NLL_4"
viz = Visdom(env = env_str)

In [24]:
win_train_loss_str = "The Loss of BATCH in the Training Data"
win_vali_loss_str = "The Loss in the Vali Data"
win_train_epoch_loss_str = "The Loss of EPOCH in the Training Data"

def draw_loss(X_step, loss, win_str):
    viz.line(X = [X_step], Y = [loss],win=win_str, update="append",
        opts= dict(title=win_str))


## 5.5 plot the metric

In [25]:
win_vali_metric_str = "The NLL of ALL vali data"
def draw_metric(X_step, total_vali_metric):

    viz.line(X = [X_step], Y = [total_vali_metric],win=win_vali_metric_str, update="append", opts= dict(title=win_vali_metric_str, legend=['pred','GT-1','GT-2'], showlegend=True,xlabel="epoch", ylabel="NLL"))


# 6 Training
## 6.1 Init

In [26]:
# torch.autograd.set_detect_anomaly(True)

mlp = MLP_1_1()
# mlp = MLP_1_2()
mlp = mlp.to(device=device)

summary(mlp, (1,8))
# mlp.half()
# mlp.block_alpha.register_forward_hook(hook_forward_fn)
# mlp.block_labda.register_forward_hook(hook_forward_fn)

# Init the params
# mlp = model_param_init(mlp)

# Save the init params
# torch.save(mlp.state_dict(), 'mlp_init.pth')

# Load the saved model
# model_path_WD = "mlp_train=300_WD.pth"
# model_path_init = 'mlp_init.pth'
# model_path_LSE = "mlp_train=300_LSE.pth"
# model_path_MLE = "mlp_train=300_MLE.pth"
model_path_MLE = "NN_params_infer_seed=26epoch=0.pth"
#
model_data = torch.load(model_path_MLE)
mlp.load_state_dict(model_data)

############ learning rate strategy ############

alpha_params = list(map(id,mlp.block_alpha.parameters()))
labda_params = list(map(id,mlp.block_labda.parameters()))

params_id = alpha_params + labda_params
other_params = filter(lambda p: id(p) not in params_id, mlp.parameters())
params = [{'params':other_params},
        {'params':mlp.block_alpha.parameters(),'lr':opt.lr_for_alpha},
        {'params':mlp.block_labda.parameters(),'lr':opt.lr_for_labda}]
optimizer = torch.optim.AdamW(params, lr=opt.learning_rate,weight_decay=opt.weight_decay)

# 2.
# Or set lr decay
# StepLR为步进，每step_size个epoch，lr*gamma
# scheduler  = torch.optim.lr_scheduler.StepLR(optimizer,step_size=3,gamma=opt.StepLR_gamma)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.95, last_epoch=-1)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.7, patience=5, verbose=False, threshold=1e-3, threshold_mode='abs', cooldown=0, min_lr=1e-7, eps=1e-7)
# threshold:只关注超过阈值的显著变化；cooldown：触发一次条件后，等待一定epoch再进行检测，避免lr下降过速；


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm1d-1                    [-1, 8]              16
            Linear-2                    [-1, 1]               9
              Tanh-3                    [-1, 1]               0
            Linear-4                    [-1, 1]               9
              ReLU-5                    [-1, 1]               0
Total params: 34
Trainable params: 34
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [27]:
# writer = SummaryWriter(log_dir="logs-MLP/"+opt.logs_str,flush_secs=60)

# Init the vis win
viz.line(X = [0.],Y = [0.], env=env_str, win=win_train_loss_str, opts= dict(title=win_train_loss_str))
viz.line(X = [0.],Y = [0.], env=env_str, win=win_train_epoch_loss_str, opts= dict(title=win_train_epoch_loss_str))


# mlp.eval()
# with torch.no_grad():
#     total_vali_metric, GT_metric = validate(mlp,val_loader,opt.N_gaussians, opt.MIN_LOSS,device)
#     draw_metric(0, total_vali_metric.cpu(), GT_metric)
# plot_net(writer,mlp,torch.randn((2,3,300),device=device))
# print("Done")

'The Loss of EPOCH in the Training Data'

In [99]:
mlp.eval()
with torch.no_grad():
    #Compute the metric
    total_test_metric, GT_metric,_ = validate_params(mlp,test_loader,opt.MIN_LOSS,device)
    print(f"========== IN Test dataset, NN Model:  {total_test_metric.detach().cpu().numpy()} ==========")
    print(f"========== IN Test dataset, the GTs: {GT_metric.detach().cpu().numpy()} ==========")
    print(f"========== The diff of metric: NN Model - GT-2(common): {GT_metric.detach().cpu().numpy()[0,1] - total_test_metric.detach().cpu().numpy()} ==========")


RuntimeError: shape '[1, 2]' is invalid for input of size 5

## 6.3 START HERE


In [28]:
# filename = "../log_file.txt"
# f = open(filename,'w')
total_train_step = 0
EPOCH_NUM = 20

mlp.train()

for epoch in range(1,EPOCH_NUM):

    epoch_train_loss = 0
    # plot_conv(writer,mlp,epoch)
    print(f"========== Now this is EPOCH {epoch} ,lr is {scheduler.get_last_lr()}==========")
    for batch_id, data in enumerate(train_loader):

        input_data, _,target_loss, _, _, _= data

        # print(f"---- {batch_id} batch----")
        # Do the inference
        input_data = input_data.to(device)
        target_loss = target_loss.to(device)
        # target_params_data = target_params_data.to(device)


        with autocast():
            alpha, labda = mlp(input_data)
            # Cal the MLE loss and draw the distrb.
            loss = loss_fn_3(input_data, alpha, labda, target_loss, opt.MIN_LOSS, device)

        epoch_train_loss += loss.detach().cpu()     # 能放在cpu上就不放在gpu上
        # print("loss.detach().item():",loss.detach().item())
        draw_loss(total_train_step, loss.detach().cpu(),win_train_loss_str)

        optimizer.zero_grad()

        # backward之前retain_graph才有用！
        # alpha.retain_grad()
        # labda.retain_grad()

        # 执行完backward之后才会register tensors的hooks
        # loss.backward()
        # mlp.block_alpha.register_full_backward_hook(hook_backward_fn)
        # mlp.block_labda.register_full_backward_hook(hook_backward_fn)

        # 对所有的梯度乘以一个clip_coef = max_norm/total_norm,
        # 因此max_norm越大，对于梯度爆炸的解决越柔和

        # alpha.register_hook(save_grad("alpha"))
        # labda.register_hook(save_grad("labda"))

        # print(f"alpha.grad = {alpha.grad}, labda.grad = {labda.grad}")
        # torch.nn.utils.clip_grad_norm_(mlp.parameters(), max_norm=50, norm_type=2)
        # optimizer.step()
        # mlp.block_alpha.register_full_backward_hook(hook_backward_fn)
        # mlp.block_labda.register_full_backward_hook(hook_backward_fn)
        # print("alpha.grad:",alpha.grad)
        # print("labda.grad:",labda.grad)

        # 如果unscale_()没有被明确调用，梯度将在step()过程中被自动取消缩放。
        # scaler.unscale_(optimizer)
        scaler.scale(loss).backward()  # scaler实现的反向误差传播
        scaler.step(optimizer)  # 优化器中的值也需要放缩
        scaler.update()  # 更新scaler
        # torch.nn.utils.clip_grad_norm_(mlp.parameters(), max_norm=50, norm_type=2)

        total_train_step += 1

        del loss
        del data
        torch.cuda.empty_cache()        # 会变慢不少
    ########### Do validation

    # Plot the loss in this EPOCH
    print(f"========== IN EPOCH {epoch} the total loss is {epoch_train_loss} ==========")
    # print(f"========== IN EPOCH {epoch} the vali NLL loss is {total_vali_metric.detach().cpu().numpy()} ==========")

    # Plot loss of this EPOCH
    draw_loss(epoch, epoch_train_loss,win_train_epoch_loss_str)
    # writer.add_scalars("Loss of EPOCH",{"Train":epoch_train_loss},epoch)
    # scheduler.step(total_vali_metric)
    scheduler.step()
    # 每跑完一个epoch，存一下
    model_params_MLP = "NN_params_infer_seed=" + str(seed) +"epoch="+str(epoch)+".pth"
    torch.save(mlp.state_dict(), model_params_MLP)

    # if epoch%3==0:
        # Record the weight
    # plot_alpha(writer,mlp,epoch,opt.tag_str)
    # plot_labda(writer,mlp,epoch,opt.tag_str)
    torch.cuda.empty_cache()        # 会变慢不少

# f.close()
# writer.close()
print("Done")

========== Now this is EPOCH 1 ,lr is [0.05, 0.05, 0.05]==========
========== IN EPOCH 1 the total loss is 10737239.0 ==========
========== Now this is EPOCH 2 ,lr is [0.0475, 0.0475, 0.0475]==========


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.35 GiB (GPU 0; 8.00 GiB total capacity; 3.98 GiB already allocated; 2.23 GiB free; 4.01 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

- save

In [27]:
torch.cuda.empty_cache()

In [160]:
model_params_MLP = "NN_params_infer_seed=" + str(seed) +".pth"
torch.save(mlp.state_dict(), model_params_MLP)

# viz.delete_env("001_test")

# 7. Load and test
## 7.1 load and init

In [101]:
Model_name = "Conv4_N_g=2"

model_params_MLP = "mlp_train=8_MLP_nll_2.pth"

mlp = MLP_1_1()
model_data = torch.load(model_params_MLP)
mlp.load_state_dict(model_data)
mlp = mlp.to(device=device)

In [180]:
mlp.eval()
with torch.no_grad():
    #Compute the metric
    total_test_metric, GT_metric,_ = validate_params(mlp,test_loader,opt.MIN_LOSS,device)
    print(f"========== IN Test dataset, NN Model:  {total_test_metric.detach().cpu().numpy()} ==========")
    print(f"========== IN Test dataset, the GTs: {GT_metric.detach().cpu().numpy()} ==========")
    print(f"========== The diff of metric: NN Model - GT-2(common): {GT_metric.detach().cpu().numpy()[0,1] - total_test_metric.detach().cpu().numpy()} ==========")


========== IN Test dataset, NN Model:  6.756695747375488 ==========
========== IN Test dataset, the GTs: [[ 6.7116656  8.723391   7.5438957 -0.8322304 -2.0117228]] ==========
========== The diff of metric: NN Model - GT-2(common): 1.9666948318481445 ==========


# 8. Generate input data